<a href="https://colab.research.google.com/github/Arumat546/nico_cls/blob/main/json_to_csv(SQL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/Google Colab/niconico_dataset_add_smid

/content/drive/My Drive/Google Colab/niconico_dataset_add_smid


## **SQLite格納**

In [ ]:
#jsonの読み込みと一覧表示例
import json
import gzip

with gzip.open("3000.json.gz", 'r') as f
     for line in f:
        jsn = json.loads(line)
        print(jsn)


In [ ]:
#json→csv変換
import pandas as pd
import json
import glob

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/video_dst" # use your path
all_files = glob.glob(path + "/*.json.gz")

li = []

for filename in all_files:
    df = pd.read_json(filename, compression='infer',lines=True)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

frame.to_csv('video.csv', columns=['video_id','category'], index=False)

In [ ]:
#sqliteデータベースに格納(動画csvファイル)
import pandas as pd
import sqlite3
import json
import glob

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid" # use your path
all_files = glob.glob(path + "/video3.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

conn = sqlite3.connect('2525.db')
c = conn.cursor()
df.to_sql('video', conn, if_exists='append')

conn.commit()
conn.close()

In [8]:
#sqliteデータベースに格納(コメントjsonファイル)改良版
import pandas as pd
import sqlite3
import json
import glob

conn = sqlite3.connect('2525.db')
c = conn.cursor()

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/comment_dst" # use your path
all_files = glob.glob(path + "/333[0-9].json.gz") #降順でいれること！


for filename in all_files:
    df = pd.read_json(filename, compression='infer', orient='records',lines=True)
    df.to_sql('comment4', conn,if_exists='append')

conn.commit()
conn.close()

In [3]:
#commentテーブル結合
import sqlite3

i=0
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("INSERT INTO all_comment (content, video_id) SELECT content,video_id FROM comment4")

con.commit()
con.close()

In [21]:
#一つずつ格納(コメントファイル)
import pandas as pd
import sqlite3
import json

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/comment_dst" # use your path

df = pd.read_json(path + "/3313.json.gz", compression='infer',lines=True) #降順でいれること！

conn = sqlite3.connect('2525.db')
c = conn.cursor()
df.to_sql('comment5', conn,if_exists='append')
conn.commit()
conn.close()

## **SQL操作**

In [37]:
#データの削除（カテゴリーがnullのものをすべて削除）
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("delete from video where category is null")

con.commit()
con.close()

In [ ]:
#列の値の種類抽出
import sqlite3
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select category from video group by category")
for row in c: # rowはtuple
    print(row)

con.commit()
con.close()

In [20]:
#ロー数確認
import sqlite3
conn = sqlite3.connect('2525.db')
conect = conn.cursor()

#SQL 設定　　テーブル名：table_ubuntu
sql = 'SELECT count(*) FROM ' + 'comment5'
#SQLを実行し、レコード数を得る
conect.execute(sql)
result = conect.fetchall()
#結果は リスト、タプル型で出力される => [(5,)]
print (result)

record_max = result[0][0]
print ('登録されている総レコード数 ==> ', record_max)

[(6964288,)]
登録されている総レコード数 ==>  6964288


In [2]:
#テーブル一覧取得
import sqlite3
con = sqlite3.connect("2525.db")
cursor = con.cursor()
cursor.execute("select * from sqlite_master where type='table'")
for x in cursor.fetchall():
    print(x)

con.close()

('table', 'video', 'video', 2, 'CREATE TABLE "video" (\n"index" INTEGER,\n  "video_id" TEXT,\n  "category" TEXT\n)')
('table', 'comment', 'comment', 192078, 'CREATE TABLE "comment" (\n"index" INTEGER,\n  "date" TIMESTAMP,\n  "vpos" INTEGER,\n  "content" TEXT,\n  "command" TEXT,\n  "video_id" TEXT\n)')
('table', 'comment2', 'comment2', 247673, 'CREATE TABLE "comment2" (\n"index" INTEGER,\n  "date" TIMESTAMP,\n  "vpos" INTEGER,\n  "content" TEXT,\n  "command" TEXT,\n  "video_id" TEXT\n)')
('table', 'comment3', 'comment3', 411441, 'CREATE TABLE "comment3" (\n"index" INTEGER,\n  "date" TIMESTAMP,\n  "vpos" INTEGER,\n  "content" TEXT,\n  "command" TEXT,\n  "video_id" TEXT\n)')
('table', 'all_comment', 'all_comment', 635050, 'CREATE TABLE all_comment(content, video_id)')
('table', 'limcom', 'limcom', 872045, 'CREATE TABLE limcom(content, video_id)')
('table', 'comment4', 'comment4', 868241, 'CREATE TABLE "comment4" (\n"index" INTEGER,\n  "date" TIMESTAMP,\n  "vpos" INTEGER,\n  "content" TEXT

In [ ]:
#カラム名取得
conn = sqlite3.connect("2525.db")
c = conn.execute('select * from limi_comment')
names = list(map(lambda x: x[0], c.description))
print(names)
conn.close()

['content', 'video_id', 'count']


In [2]:
#テーブル削除
import sqlite3
con = sqlite3.connect('2525.db')
cur = con.cursor()

sql = "drop table comment4"
cur.execute(sql)
 
con.close()

In [ ]:
#内部結合
import sqlite3
i=0
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select content,category from comment inner join video on comment.video_id=video.video_id where length(content)>=10 and length(content)<=20 and (category='アイドルマスター'or category='東方')")
for row in c: # rowはtuple
    print(row[0], row[1])
    i+=1

print(i)
con.commit()
con.close()

In [9]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [ ]:
#tsv出力
import pandas as pd
import sqlite3

i=0
df = pd.DataFrame(index=[], columns=['content','category'])

con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select content,category from comment inner join video on comment.video_id=video.video_id where length(content)>=8 and category='アイドルマスター'")
for row in c: # rowはtuple
    #print(row[0], row[1])
    df.loc[i]=[row[0], row[1]]
    i+=1

df.head()
#データをシャッフル
df = df.sample(frac=1, random_state=121).reset_index(drop=True)


#取り出すデータ数指定
df[:2000].to_csv('/content/imas2000.tsv', sep='\t', index=False, header=None)
#print(df[:len].shape)
#df.to_csv('/content/entame.tsv', sep='\t', index=False)
con.commit()
con.close()

## **実験場**

In [12]:
#投稿時刻確認
import sqlite3

connection = sqlite3.connect('2525.db')
cursor = connection.cursor()

cursor.execute("select max(date), min(date) from comment4")
for row in cursor: # rowはtuple
  print(row)
connection.commit()
connection.close()

('2018-11-07 19:59:42', '2018-06-01 14:03:59')


In [ ]:
#各コメント数集計
import sqlite3
import csv
i=0
com=[]
num=[]

connection = sqlite3.connect('2525.db')
cursor = connection.cursor()

cursor.execute("select content, count(*) as namecount from comment group by content having(count(*)>1000) order by content DESC")
for row in cursor: # rowはtuple

    print(row[0], row[1])
    com[i]=row[0]
    num[i]=row[1]
    i+=1
#print(i)

connection.commit()
connection.close()

In [ ]:
import sqlite3 
#特定カテゴリのデータ数確認
i=0
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select * from video where category='音楽'")
for row in c: # rowはtuple
    #print(row)
    i+=1

print(i)
con.commit()
con.close()


1083519


In [23]:
#動画ごとのコメントを確認
import sqlite3

con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("SELECT content FROM all_comment WHERE video_id = 'sm34103741'")
for row in c: # rowはtuple
    print(row)
con.commit()
con.close()

('うぽつ～',)
('うぽつー',)
('Gj',)
('いいね',)
('好き',)
('うぽつ',)
('萩原さんは才能が開花しつつあるように思える',)
('888888888888888888888888',)
('「物置」',)
('「作詞作曲:萩原九厘」',)
('歌:初音ミク',)
('ボカロメタルみたいに',)
('すんごいエモい',)


In [10]:
#selectしてinsert
import sqlite3

i=0
#df = pd.DataFrame(index=[], columns=['num'])
con = sqlite3.connect("2525.db")
c = con.cursor()
#コメント数が10以上100未満の動画データのみ別のテーブルにインサート
#c.execute("INSERT INTO limi_comment (content, video_id) SELECT content,video_id FROM all_comment GROUP BY video_id HAVING count(video_id)>=11 AND count(video_id)<=38")
c.execute("INSERT INTO limcom SELECT content,video_id FROM comment4 WHERE video_id IN (SELECT video_id FROM comment4 GROUP BY video_id HAVING count(*)>=11 AND count(*)<=38)")
for row in c: # rowはtuple
    #print(row)
    #df.loc[i]=[row[1]]
    i+=1

#df.to_csv('/content/comment_num.csv', index=False, header=None)
con.commit()
con.close()

In [1]:
%cd /content/drive/My Drive/Google Colab/niconico_dataset_add_smid

/content/drive/My Drive/Google Colab/niconico_dataset_add_smid


In [92]:
#動画IDごとのコメント数取得
import sqlite3
import pandas as pd

i=0
num1=0
num2=0
num3=0
num4=0
num5=0
num6=0
num7=0
num8=0
num9=0
num10=0
numd=0


#df = pd.DataFrame(index=[], columns=['num'])
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("SELECT count(*) AS COUNT FROM all_comment GROUP BY all_comment.video_id ORDER BY COUNT DESC")
print("a")
for row in c: # rowはtuple
    if row[0] == 31:
      num1=num1+1
    elif row[0] == 32:
      num2=num2+1
    elif row[0] == 33:
      num3=num3+1
    elif row[0] == 34:
      num4=num4+1
    elif row[0] == 35:
      num5=num5+1
    elif row[0] == 36:
      num6=num6+1
    elif row[0] == 37:
      num7=num7+1
    elif row[0] == 38:
      num8=num8+1
    elif row[0] == 39:
      num9=num9+1
    elif row[0] == 40:
      num10=num10+1
    elif row[0] > 35:
      numd=numd+1
    
    i+=1
print(num1)
print(num2)
print(num3)
print(num4)
print(num5)
print(num6)
print(num7)
print(num8)
print(num9)
print(num10)
print(numd)
#df.to_csv('/content/comment_num.csv', index=False, header=None)
con.commit()
con.close()

a
1141
1012
940
1024
949
0
0
0
0
0
52505


In [ ]:
#特定文字数以上のレコードの取得（select)
con = sqlite3.connect("nico.db")
c = con.cursor()
c.execute("select content,video_id from comment where length(content)>=20 and length(content)<=22")#5文字以上8文字以内
for row in c: # rowはtuple
    print(row)
con.commit()
con.close()

In [6]:
#コメントデータに含まれる動画数
import sqlite3
i=0
connection = sqlite3.connect('2525.db')
cursor = connection.cursor()

cursor.execute("select count(*) AS COUNT from limit_comment group by video_id")
for row in cursor: # rowはtuple
   # print(row)
   i=i+1
print(i)
connection.commit()
connection.close()

202486


In [20]:
import sqlite3

connection = sqlite3.connect('2525.db')
cursor = connection.cursor()

i=0
cursor.execute("select count(*) as COUNT from limcom group by video_id")
for row in cursor:
  i=i+1

print(i+1)

connection.commit()
connection.close()

50242


In [11]:
#テーブル作成
import sqlite3

connection = sqlite3.connect('2525.db')
cursor = connection.cursor()

cursor.execute("create table limcom(content, video_id);")

connection.commit()
connection.close()